Let’s begin Stage 1: The Core Agent Node — our foundation for understanding how LangGraph executes agent logic.

Stage 1: The Core Agent Node — Your First LangGraph Agent

🎯 Goal

- Build a minimal LangGraph agent that:

- Takes a user query (e.g., “Explain transformers in one line”)

- Uses an LLM to respond

- Passes messages through a state graph (no tools, no memory yet)

🧠 Concepts Introduced

- StateGraph: defines the overall workflow (nodes + edges)

- State: carries messages or intermediate data between nodes

- Node function: acts as an “agent brain” — it receives the state and returns updates

- Graph execution: LangGraph runs your defined graph (with or without streaming)

In [5]:
from dotenv import load_dotenv
from langgraph.graph import StateGraph, MessagesState, END
from langchain_openai import ChatOpenAI
load_dotenv("../../config/local.env")

True

In [6]:
# 1️⃣ Define the LLM
llm = ChatOpenAI(model="gpt-4o-mini")

In [7]:

# 2️⃣ Define the agent node (a function that processes state)
def agent_node(state: MessagesState):
    user_message = state["messages"][-1].content
    print(f"🧠 Agent received: {user_message}")
    
    # Get LLM response
    response = llm.invoke(user_message)
    
    # Return updated state
    return {"messages": state["messages"] + [response]}


In [8]:

# 3️⃣ Build the graph
builder = StateGraph(MessagesState)
builder.add_node("agent", agent_node)
builder.set_entry_point("agent")   # where execution starts
builder.add_edge("agent", END)     # no next node, ends here

# 4️⃣ Compile the graph
graph = builder.compile()

# 5️⃣ Run the agent
result = graph.invoke({"messages": [{"role": "user", "content": "Explain transformers in one line"}]})


🧠 Agent received: Explain transformers in one line


In [9]:

# 6️⃣ Print output
print("🤖 Agent says:", result["messages"][-1].content)


🤖 Agent says: Transformers are a type of neural network architecture designed for processing sequential data, using self-attention mechanisms to weigh the importance of different elements in the input sequence.


### Explanation

- MessagesState is a pre-defined state type in LangGraph for chat-based messages.

- The function agent_node acts like a thinking unit — it reads from state, processes, and updates it.

- StateGraph defines your workflow topology — here, a single node.

- .invoke() executes the workflow once. Later, we’ll add .stream() for real-time updates.